# 01 - Interview Question Generation
`Author: Abdlazeez Jimoh`

In [1]:
from openai import OpenAI
import json
from pprint import pprint
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [2]:
client = OpenAI()

In [5]:
class BaseAgent:
    def __init__(self):
        self.client = OpenAI()
        self.system_prompt = None
        self.user_prompt = None

    def __call__(self, *args, **kwargs):
        raise NotImplementedError

    def run(self, *args, **kwargs):
        raise NotImplementedError


class QuestionGeneratorAgentV1(BaseAgent):
    def __init__(self):
        super().__init__()
        self.system_prompt = """You are a non-technical interviewer that generate at least 4 interview questions across the following categories:
- personal
- role-specific
- behavioural
- situational

Make sure there is a question for each category.

You answer strictly as single JSON string. Don't include any other verbose texts and don't include the markdown syntax anywhere.

JSON format:
[
    {"question": "<question>", "question_type": "<question_type>"}
]"""
        self.user_prompt = "{description}"

    def __call__(self, description: str, n_questions: int = 5) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 5.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 5) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 5.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list: A list of generated interview questions.
        """

        try:
            output = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt,
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(description=description),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )

            questions = json.loads(output.choices[0].message.content or "[]")

            return questions
        except Exception as e:
            print(e)
            return []


class QuestionGeneratorAgentV2(BaseAgent):
    def __init__(self):
        super().__init__()
        self.system_prompt = """You are a non-technical interviewer that interviews across the following categories:
- personal
- role-specific
- behavioural
- situational

Your questions should be based on the provided candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "question_type": "personal"}},
    {{"question": "<role_specific_question>", "question_type": "role-specific"}},
    {{"question": "<behavioural_question>", "question_type": "behavioural"}},
    {{"question": "<situational_question>", "question_type": "situational"}},
    ...
]"""

        self.user_prompt = "{description}"

    def __call__(self, description: str, n_questions: int = 4) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            list: A list of generated interview questions.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> list[dict[str, str]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            list: A list of generated interview questions.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output = client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt.format(n_questions=n_questions),
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(description=description),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            questions = json.loads(output.choices[0].message.content or "[]")

            return questions
        except Exception as e:
            print("Error:", e)
            return []

In [6]:
question_generator = QuestionGeneratorAgentV1()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job."
)

pprint(questions)

[{'question': 'Tell me about a time when you had to quickly learn a new '
              'technology or programming language. How did you approach the '
              'challenge?',
  'question_type': 'behavioural'},
 {'question': 'Can you walk me through a recent project you worked on and the '
              'role you played in its development?',
  'question_type': 'role-specific'},
 {'question': 'How do you prioritize your tasks and manage your time '
              'effectively, especially in a fast-paced startup environment?',
  'question_type': 'situational'},
 {'question': 'What do you consider to be your greatest technical strength, '
              'and how does it contribute to your effectiveness as a software '
              'engineer?',
  'question_type': 'personal'},
 {'question': 'Imagine you have to work on a project with a tight deadline and '
              'limited resources. How would you approach this situation to '
              'ensure successful delivery?',
  'question

In [7]:
question_generator2 = QuestionGeneratorAgentV2()
questions2 = question_generator2.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job.",
)

pprint(questions2)

[{'question': 'What inspired you to pursue a career in software engineering?',
  'question_type': 'personal'},
 {'question': 'Can you discuss a project where you had to work with limited '
              'resources and tight deadlines?',
  'question_type': 'role-specific'},
 {'question': 'Tell me about a time when you had to adapt to a significant '
              'change in a project. How did you handle it?',
  'question_type': 'behavioural'},
 {'question': "You're working on a project with a tight deadline and one of "
              'your team members falls ill. How would you handle this '
              'situation?',
  'question_type': 'situational'}]


In [8]:
question_generator2 = QuestionGeneratorAgentV2()
questions3 = question_generator2.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
)

pprint(questions3)

[{'question': 'What inspired you to pursue a career in data science?',
  'question_type': 'personal'},
 {'question': 'Can you explain how you implemented the recommendation engine '
              'for the e-commerce company?',
  'question_type': 'role-specific'},
 {'question': 'Tell me about a time when you had to resolve a conflict within '
              'your team while working on a project.',
  'question_type': 'behavioural'},
 {'question': 'How would you approach a situation where the data for a project '
              'is incomplete or unreliable?',
  'question_type': 'situational'}]
